In [1]:
%cd ..

/environment/Projects/dl


In [2]:
! python -m pip install deps/light-torch/

Processing ./deps/light-torch
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for light-torch: filename=light_torch-0.0.1-py3-none-any.whl size=6012 sha256=0720e01589dbba642c2c88e82b39d2680668a5df03264a3b9e5a014c0f4e855f
  Stored in directory: /home/stefan-cristianhantig/.cache/pip/wheels/76/4b/a5/ce53c36ade717c024783dce5309d843ab5a592d06a292dafc2
Successfully built light-torch
  Attempting uninstall: light-torch
    Found existing installation: light-torch 0.0.1
    Uninstalling light-torch-0.0.1:
      Successfully uninstalled light-torch-0.0.1


In [8]:
from light_torch.module import Module
from src.object_detection.yolov1 import YoloV1Backbone, YoloV1, ObjectDetectionModule
from src.classification.models.resnet import Resnet
from torchsummary import summary
import json
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data.dataset import Dataset
from functools import lru_cache
from torchvision.transforms import Compose
from IPython.core.debugger import set_trace
from src.object_detection.data.coco import CocoDetectionDataset, make_id2category_map, collect_annotations
from src.object_detection.utils.visualization import draw_bbox_xyhw, draw_rect_xywh_, draw_grid_, draw_boxes_tlbr_
import torch
from src.transform import to_rgb, to_torch, resize
from functools import partial
from torch.nn import functional as F
from collections import defaultdict
from src.object_detection.yolov1 import YoloV1Loss
from collections import defaultdict
from light_torch.train import Trainer
from torch.utils.data import DataLoader
from src.object_detection.data.utils import pad_collate
from src.torchx import target_to_torch

In [3]:
TRAIN_ANNOTATION_FILE = "resources/object_detection/annotations/instances_train2014.json"
VAL_ANNOTATION_FILE = "resources/object_detection/annotations/instances_val2014.json"

In [4]:
with open(TRAIN_ANNOTATION_FILE, "r") as json_file:
    train_coco = json.load(json_file)
    
with open(VAL_ANNOTATION_FILE, "r") as json_file:
    val_coco = json.load(json_file)

In [5]:
TRAIN_IMAGES_PATH = Path("./resources/object_detection/train2014/")
VAL_IMAGES_PATH = Path("./resources/object_detection/val2014/")

In [6]:
img_transform = Compose([
    to_torch,
    
])

In [10]:
def select_categories(annotation_dict, categories=None):
    return annotation_dict["category_id"] in categories
    

In [11]:

coco_train_ds = CocoDetectionDataset(
    train_coco, 
    TRAIN_IMAGES_PATH,
    transform=to_torch,
    target_transform=target_to_torch,
    select_annotation=partial(select_categories, categories=[78, 79]),
    
)

val_train_ds = CocoDetectionDataset(
    val_coco, 
    VAL_IMAGES_PATH,
    transform=to_torch,
    target_transform=target_to_torch,
    select_annotation=partial(select_categories, categories=[78, 79]),
    
)
train_dl = DataLoader(coco_train_ds, collate_fn=pad_collate, batch_size=2, shuffle=True)
val_dl = DataLoader(val_train_ds, collate_fn=pad_collate, batch_size=2, shuffle=True)

In [12]:
#resnet = Resnet()
yolo = YoloV1(coco_train_ds.categories_num)

In [13]:
yolov1_loss = YoloV1Loss()

In [20]:
from torchvision import transforms as TF

In [26]:
obj_detection = ObjectDetectionModule(
    yolo, 
    yolov1_loss, 
    coco_train_ds.id2name_map, 
    iou_threshold=0.2,
    transform=Compose([TF.Normalize((0, 0, 0), (1, 1, 1))]),
)

In [27]:
optimizer = torch.optim.SGD(obj_detection.parameters(), lr=0.001)

In [28]:
trainer = Trainer(
    obj_detection, 
    epochs=10,
    accumulation=4,
    optimizer=optimizer,
    name="yolov1",
)

In [29]:
trainer.train()

Trainer(
  (module): ObjectDetectionModule(
    (model): YoloV1(
      (backbone): YoloV1Backbone(
        (st_conv): Conv2dSamePadding(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
        (st_maxpol): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
        (net): Sequential(
          (conv2): Conv2dSamePadding(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (activation2): LeakyReLU(negative_slope=0.01)
          (pool2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
          (identity_conv31): Conv2dSamePadding(192, 128, kernel_size=(1, 1), stride=(1, 1))
          (conv31): Conv2dSamePadding(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (activation31): LeakyReLU(negative_slope=0.01)
          (identity_conv32): Conv2dSamePadding(256, 256, kernel_size=(1, 1), stride=(1, 1))
          (conv32): Conv2dSamePadding(256, 512, kernel_size=(3, 3), stride=(1, 1), pad

In [ ]:
trainer.fit(train_dl, val_data=val_dl)

Train epoch  001:   0%|          | 0/1227 [00:00<?, ?it/s]

STAGE TRAIN: Epoch:  001
{'box_loss': 24.84475357599478, 'classification_loss': 7.072964195663014, 'confidence_loss': 0.0016038249361499967, 'loss': 31.919321596593843}


Val epoch  001:   0%|          | 0/603 [00:00<?, ?it/s]

STAGE VAL: Epoch:  001
{'box_loss': 25.032044027781435, 'classification_loss': 6.818803132766515, 'confidence_loss': 0.0004256292899296278, 'loss': 31.851272789837864, 'mAP_0.5': 3.80087255292722e-05, 'mAP_0.55': 2.8323926818561035e-05, 'mAP_0.6': 2.908595302112031e-05, 'mAP_0.65': 5.9153984264796995e-06, 'mAP_0.7': 5.9340665861611635e-06, 'mAP_0.75': 1.948246772729221e-06, 'mAP_0.8': 9.741233863646105e-07, 'mAP_0.85': 9.741233863646105e-07, 'mAP_0.9': 5.902933344666966e-07, 'mAP_0.95': 0.0}


Train epoch  002:   0%|          | 0/1227 [00:00<?, ?it/s]

In [51]:
with torch.no_grad():
    obj_detection.val_step(batch)

In [52]:
obj_detection.on_val_end()

In [54]:
obj_detection.get_batch_log()

{'box_loss': 49.54795295827014,
 'classification_loss': 11.735513687133789,
 'confidence_loss': 12.811930656433105,
 'loss': 74.09539730183704,
 'mAP_0.5': 0.0,
 'mAP_0.55': 0.0,
 'mAP_0.6': 0.0,
 'mAP_0.65': 0.0,
 'mAP_0.7': 0.0,
 'mAP_0.75': 0.0,
 'mAP_0.8': 0.0,
 'mAP_0.85': 0.0,
 'mAP_0.9': 0.0,
 'mAP_0.95': 0.0}

In [53]:
obj_detection.get_epoch_log()

{'box_loss': 49.54795295827014,
 'classification_loss': 11.735513687133789,
 'confidence_loss': 12.811930656433105,
 'loss': 74.09539730183704,
 'mAP_0.5': 0.0,
 'mAP_0.55': 0.0,
 'mAP_0.6': 0.0,
 'mAP_0.65': 0.0,
 'mAP_0.7': 0.0,
 'mAP_0.75': 0.0,
 'mAP_0.8': 0.0,
 'mAP_0.85': 0.0,
 'mAP_0.9': 0.0,
 'mAP_0.95': 0.0}